In [5]:
import pickle
import os
import pandas as pd
from matplotlib import pyplot as plt
os.chdir(os.path.expanduser('~/wcEcoli/'))
# noinspection PyUnresolvedReferences
import numpy as np
import plotly.graph_objects as go
from models.ecoli.analysis import cohortAnalysisPlot
from wholecell.analysis.analysis_tools import (exportFigure,
	read_bulk_molecule_counts, read_stacked_bulk_molecules, read_stacked_columns)
from wholecell.io.tablereader import TableReader
from sklearn.metrics import r2_score

In [6]:
current_sequence = "CLClim3NE1"
CLNE_sequence = "CLNE_11192024"

In [7]:
# read in the unfiltered data: 
CLClimNE_log_data_with_proteases = pd.read_csv('out/CLClim3NE1/wildtype_000000/cohort_average_monomer_count_data/unfiltered_data/log_data/LogAvgProteinCounts_startGen_2.csv')

# remove the last three characters from each entry in the Monomer ID column: 
CLClimNE_log_data_with_proteases['Monomer ID'] = CLClimNE_log_data_with_proteases['Monomer ID'].str[:-3]
CLClimNE_log_data_with_proteases

,Monomer ID,Log10 Average Monomer Counts
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,2.239675
1,1-PFK-MONOMER,1.747600
2,2-DEHYDROPANTOATE-REDUCT-MONOMER,2.321794
3,2-ISOPROPYLMALATESYN-MONOMER,3.482980
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,2.695617
...,...,...
4305,YTFR-MONOMER,1.180822
4306,YTFT-MONOMER,1.138950
4307,ZNUA-MONOMER,2.872238
4308,ZNUB-MONOMER,1.829640


add the gene names and the protease assignments

In [8]:
# add the gene names: 
common_names = "reconstruction/ecoli/scripts/protein_half_lives/Gupta_et_al_Clim_data/Clim_EcoCyc_monomer_ID_matches/41467_2024_49920_MOESM4_ESM_ST1_EcoCyc_monomer_ID_matches_11202024.tsv"
common_names = pd.read_csv(common_names, sep='\t', skiprows=1)
# read in the priority protease data: 
priority_substrates = "models/ecoli/analysis/local_notebooks/protease_substrate_relationships/data/priority_protease_assignments_0.tsv"
priority_substrates = pd.read_csv(priority_substrates, sep='\t', skiprows=1)
# add the CLim3 HL source: 
CLClim3NE1_HLs = "out/CLClim3NE1/kb_plot_out/protein_half_lives.tsv"
CLClim3NE1_HLs = pd.read_csv(CLClim3NE1_HLs, sep='\t')

# add the protease assginment
CLClimNE_log_data_with_proteases['protease assignment'] = "NA"
for index, row in CLClimNE_log_data_with_proteases.iterrows():
    if row['Monomer ID'] in priority_substrates['id'].values:
        CLClimNE_log_data_with_proteases.at[index, 'protease assignment'] = priority_substrates[priority_substrates['id'] == row['Monomer ID']]['protease_assignment'].values[0]
   
# add the gene symbol
CLClimNE_log_data_with_proteases['gene symbol'] = None
for index, row in CLClimNE_log_data_with_proteases.iterrows():
    if row['Monomer ID'] in common_names['Monomer ID'].values:
        CLClimNE_log_data_with_proteases.at[index, 'gene symbol'] = common_names[common_names['Monomer ID'] == row['Monomer ID']]['Common Name'].values[0]
        
# add in the HL source: 
CLClimNE_log_data_with_proteases['HL source'] = None
CLClimNE_log_data_with_proteases['HL value'] = None
for index, row in CLClimNE_log_data_with_proteases.iterrows():
    if row['Monomer ID'] in CLClim3NE1_HLs['monomer_id'].values:
        CLClimNE_log_data_with_proteases.at[index, 'HL source'] = CLClim3NE1_HLs[CLClim3NE1_HLs['monomer_id'] == row['Monomer ID']]['degradation_rate_source'].values[0]
        CLClimNE_log_data_with_proteases.at[index, 'HL value'] = CLClim3NE1_HLs[CLClim3NE1_HLs['monomer_id'] == row['Monomer ID']]['half_life_(min)'].values[0]
        
   
# add the half life source in the Clim3 data: 
# todo actually there is only so much that can be in the hover data, see if this can be added too 

CLClimNE_log_data_with_proteases

,Monomer ID,Log10 Average Monomer Counts,protease assignment,gene symbol,HL source,HL value
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,2.239675,NA,plsC,Gupta_et_al_MS_2024,501.707893
1,1-PFK-MONOMER,1.747600,NA,fruK,Gupta_et_al_MS_2024,766.256894
2,2-DEHYDROPANTOATE-REDUCT-MONOMER,2.321794,NA,panE,N_end_rule,600.0
3,2-ISOPROPYLMALATESYN-MONOMER,3.482980,NA,leuA,Gupta_et_al_MS_2024,276.924332
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,2.695617,NA,ubiE,Gupta_et_al_MS_2024,493.39705
...,...,...,...,...,...,...
4305,YTFR-MONOMER,1.180822,NA,ytfR,Gupta_et_al_MS_2024,654.149344
4306,YTFT-MONOMER,1.138950,NA,ytfT,N_end_rule,600.0
4307,ZNUA-MONOMER,2.872238,NA,znuA,Gupta_et_al_MS_2024,638.129701
4308,ZNUB-MONOMER,1.829640,NA,None,N_end_rule,600.0


In [9]:
# figure out which proteins were assigned to lon in the model: 
lon_proteins = CLClimNE_log_data_with_proteases[CLClimNE_log_data_with_proteases['protease assignment'] == 'Lon only']
lon_proteins

# figure out which proteins were assigned to clp in the model:
clp_proteins = CLClimNE_log_data_with_proteases[CLClimNE_log_data_with_proteases['protease assignment'] == 'ClpP only']

hslv_proteins = CLClimNE_log_data_with_proteases[CLClimNE_log_data_with_proteases['protease assignment'] == 'HslV only']

# figure out which proteins are additive in the model: 
additive_proteins = CLClimNE_log_data_with_proteases[CLClimNE_log_data_with_proteases['protease assignment'] == 'Additive: ClpP, Lon']

# figure out which proteins are redundant in the model: 
redundant_proteins = CLClimNE_log_data_with_proteases[CLClimNE_log_data_with_proteases['protease assignment'] == 'Additive: ClpP, Lon, HslV']



# figure out which proteins were not assigned to a protease in the model:
no_protease_proteins = CLClimNE_log_data_with_proteases[CLClimNE_log_data_with_proteases['protease assignment'] == 'NA']

# todo: note that these numbers will not add up to those in the graph below bc some proteins in the CLClim0NE model are not in the CLNE model 
#298 total 
print("Lon Proteins(" +str(len(lon_proteins))+"): ", list(lon_proteins['gene symbol'])) #14
print("ClpP Proteins(" +str(len(clp_proteins))+"): ", list(clp_proteins['gene symbol'])) #63
print("HslV Proteins(" +str(len(hslv_proteins))+"): ", list(hslv_proteins['gene symbol'])) #1
print("Additive Proteins: Lon, ClpP (" +str(len(additive_proteins))+"): ", list(additive_proteins['gene symbol'])) #79
print("Additive: Lon, ClpP, HslV Proteins(" +str(len(redundant_proteins))+"): ", list(redundant_proteins['gene symbol'])) #39
#print("No Protease Proteins(" +str(len(no_protease_proteins))+"): ", list(no_protease_proteins['gene symbol'])) # 4012

# ClpP does not have 

Lon Proteins(6):  ['ydcI', 'nemA', 'fhlA', 'metR', 'trxC', 'rpoD']
ClpP Proteins(40):  ['fadE', 'glpD', 'cysD', 'ribB', 'clpA', 'clpX', 'dksA', 'dnaB', 'dnaK', 'ftsZ', 'mukB', 'mutS', 'nfo', 'parC', 'pcnB', 'recA', 'rpsA', 'sbcC', 'uvrD', 'dps', 'def', 'phoH', 'intA', 'rutA', 'comR', 'ydhQ', 'yfcZ', 'patA', 'obgE', 'yheO', 'lldD', 'mdlB', 'oxyR', 'exuR', 'putA', 'rpoB', 'rpoC', 'rpoS', 'yheS', 'znuC']
HslV Proteins(1):  ['uhpA']
Additive Proteins: Lon, ClpP (3):  ['ibpA', 'grcA', 'dnaA']
Additive: Lon, ClpP, HslV Proteins(32):  ['aroK', 'helD', 'ligA', 'minE', 'parE', 'pspA', 'rhlB', 'srmB', 'ybaB', 'yibA', 'mazF', 'yjgA', 'yihD', 'yiiQ', 'yiaU', 'yfhH', 'erpA', 'ppiC', 'glaR', 'chaB', 'rarA', 'proQ', 'acpP', 'ycaR', 'hprR', 'murQ', 'iscR', 'kbp', 'yggX', 'glnD', 'yaeP', 'thiL']


In [10]:
# prepare the data for a plot:
CLClimNE_log_data_with_proteases = CLClimNE_log_data_with_proteases.copy()

CLNE_log_data = pd.read_csv('/Users/miagrahn/wcEcoli/out/CLNE_11192024/wildtype_000000/cohort_average_monomer_count_data/unfiltered_data/log_data/LogAvgProteinCounts_startGen_2.csv')
CLNE_log_data['Monomer ID'] = CLNE_log_data['Monomer ID'].str[:-3]

# merge the two dataframes:
CLNE_new_name = "Log10 " +CLNE_sequence +" Average Monomer Counts"
CLClimNE_new_name = "Log10 " +current_sequence +" Average Monomer Counts"

CLNE_log_data = CLNE_log_data.rename(columns={"Log10 Average Monomer Counts": CLNE_new_name})
CLClimNE_log_data_with_proteases = CLClimNE_log_data_with_proteases.rename(columns={"Log10 Average Monomer Counts": CLClimNE_new_name})
#CLNE_log_data = CLNE_log_data[["Monomer ID", "Log10 CLNE Protein Counts"]]
#CLClimNE_log_data_with_proteases = CLClimNE_log_data_with_proteases[["Monomer ID", "Log10 CLClim0NE Protein Counts"]]
CLNE_log_data = CLNE_log_data.set_index("Monomer ID")
CLClimNE_log_data_with_proteases = CLClimNE_log_data_with_proteases.set_index("Monomer ID")
CLNE_CLClimNE_log_data_proteases = CLNE_log_data.join(CLClimNE_log_data_with_proteases, how="inner")
CLNE_CLClimNE_log_data_proteases = CLNE_CLClimNE_log_data_proteases.reset_index()

# make a copy of the CLClimNE_log_data_with_proteases data:
CLClimNE_log_data_with_Lon_proteases = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_Clp_proteases = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_HslV_proteases = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_Additive_LC = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_Additive_LCH = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_no_protease_proteases = CLNE_CLClimNE_log_data_proteases.copy()

# remove the proteins that were not assigned to the protease in the model:
CLClimNE_log_data_with_Lon_proteases = CLClimNE_log_data_with_Lon_proteases[CLClimNE_log_data_with_Lon_proteases['protease assignment'] == 'Lon only']
CLClimNE_log_data_with_Clp_proteases = CLClimNE_log_data_with_Clp_proteases[CLClimNE_log_data_with_Clp_proteases['protease assignment'] == 'ClpP only']
CLClimNE_log_data_with_HslV_proteases = CLClimNE_log_data_with_HslV_proteases[CLClimNE_log_data_with_HslV_proteases['protease assignment'] == 'HslV only']
CLClimNE_log_data_with_Additive_LC_proteases = CLClimNE_log_data_with_Additive_LC[CLClimNE_log_data_with_Additive_LC['protease assignment'] == 'Additive: ClpP, Lon']
CLClimNE_log_data_with_Additive_LCH_proteases = CLClimNE_log_data_with_Additive_LCH[CLClimNE_log_data_with_Additive_LCH['protease assignment'] == 'Additive: ClpP, Lon, HslV']
CLClimNE_log_data_with_no_protease_proteases = CLClimNE_log_data_with_no_protease_proteases[CLClimNE_log_data_with_no_protease_proteases['protease assignment'] == 'NA']


In [11]:
## Create scatter plot of CLNE counts to CLClim0NE counts, but color the protiens based on their protease assignment in the CLClim0NE model
# create the plot!
# TODO: do I need to normalize the data to ? and how do I get the plot to be an exact square? and the figure title centered?
def CLNE_CLClimNE_plot_protease(fig, dataframe, CLNE_new_name, CLClimNE_new_name, protease=0):
    x = dataframe[CLNE_new_name]
    y = dataframe[CLClimNE_new_name]
    #hovertext = {dataframe["gene symbol"],dataframe["HL value"],dataframe["HL source"]}
    hovertext = dataframe.apply(lambda row: f"Common Name: {row['gene symbol']}<br>HL Value: {row['HL value']}<br>HL Source: {row['HL source']}", axis=1)

 
    # Add scatter trace
    if protease == 0: # no assignement
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"No Protease ({np.shape(dataframe)[0]})", marker=dict(color='lightseagreen', size=3, opacity=.3)))

    elif protease == 1: # ClpP and Lon
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"Additive: ClpP, Lon ({np.shape(dataframe)[0]})", marker=dict(color='cornflowerblue', size=5, opacity=.9)))
        
    elif protease == 2: # C L H
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"Additive: ClpP, Lon, HslV ({np.shape(dataframe)[0]})", marker=dict(color='orange', size=4, opacity=.7)))
        
    elif protease == 4: # lon
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"Lon ({np.shape(dataframe)[0]})", marker=dict(color='yellowgreen', size=5, opacity=.9)))
        
    elif protease == 5: # clp
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"ClpP ({np.shape(dataframe)[0]})", marker=dict(color='deeppink', size=4, opacity=.6)))
        
    elif protease == 6: # hslv
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"Hslv ({np.shape(dataframe)[0]})", marker=dict(color='brown', size=5, opacity=.9)))

# now plot!

# Create figure
fig = go.Figure()

# Add scatter plots and trendlines for each dataset
CLNE_CLClimNE_plot_protease(fig, CLClimNE_log_data_with_no_protease_proteases,CLNE_new_name, CLClimNE_new_name, 0)
CLNE_CLClimNE_plot_protease(fig, CLClimNE_log_data_with_Additive_LCH_proteases,CLNE_new_name, CLClimNE_new_name, 2)
CLNE_CLClimNE_plot_protease(fig, CLClimNE_log_data_with_Additive_LC_proteases,CLNE_new_name, CLClimNE_new_name, 1)

CLNE_CLClimNE_plot_protease(fig, CLClimNE_log_data_with_Lon_proteases,CLNE_new_name, CLClimNE_new_name, 4)
CLNE_CLClimNE_plot_protease(fig, CLClimNE_log_data_with_Clp_proteases,CLNE_new_name, CLClimNE_new_name, 5)
CLNE_CLClimNE_plot_protease(fig, CLClimNE_log_data_with_HslV_proteases,CLNE_new_name, CLClimNE_new_name, 6)

# add a y=x line
fig.add_trace(go.Scatter(x=[0, 6], y=[0, 6], mode="lines",
    line=go.scatter.Line(color="black", dash="dash"), opacity=0.2,
    name="y=x"));

# Update layout
fig.update_layout(
    title=f"Protein Counts from the New WCM plotted <br>against those from the Original WCM",
    xaxis_title=f"log10({CLNE_sequence} Protein Counts)",
    yaxis_title=f"log10({current_sequence} Protein Counts)",
    autosize=False,
    width=900,
    height=600, 
    showlegend=True,
    legend_title="Protease Assignments")


# wont open in pycharm, so save as a html:
out_pth = "models/ecoli/analysis/local_notebooks/protease_substrate_relationships/protein_count_comparisons_outputs" 
out_pth = out_pth + "/" + current_sequence + "_vs_" + CLNE_sequence + "_protein_count_comparison.html"
out_pth = os.path.expanduser(out_pth) 
output_dir = os.path.dirname(out_pth)
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it does not exist
fig.write_html(out_pth, auto_open=True) # displays the figure on a default web browser 



# now tag proteins by their degradation rate

In [19]:
# sort the data by the degradation rate:
CLNE_CLClimNE_log_data_proteases


,Monomer ID,Log10 CLNE_11192024 Average Monomer Counts,Log10 CLClim3NE1 Average Monomer Counts,protease assignment,gene symbol,HL source,HL value
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,2.248895,2.239675,NA,plsC,Gupta_et_al_MS_2024,501.707893
1,1-PFK-MONOMER,1.829031,1.747600,NA,fruK,Gupta_et_al_MS_2024,766.256894
2,2-DEHYDROPANTOATE-REDUCT-MONOMER,2.155309,2.321794,NA,panE,N_end_rule,600.0
3,2-ISOPROPYLMALATESYN-MONOMER,3.603593,3.482980,NA,leuA,Gupta_et_al_MS_2024,276.924332
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,2.619919,2.695617,NA,ubiE,Gupta_et_al_MS_2024,493.39705
...,...,...,...,...,...,...,...
4305,YTFR-MONOMER,1.591232,1.180822,NA,ytfR,Gupta_et_al_MS_2024,654.149344
4306,YTFT-MONOMER,1.569764,1.138950,NA,ytfT,N_end_rule,600.0
4307,ZNUA-MONOMER,2.792515,2.872238,NA,znuA,Gupta_et_al_MS_2024,638.129701
4308,ZNUB-MONOMER,1.856827,1.829640,NA,None,N_end_rule,600.0


In [15]:

# make a copy of the CLClimNE_log_data_with_proteases data:
CLClimNE_log_data_with_25_below = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_25_50 = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_50_75 = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_75_100 = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_100_125 = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_125_150 = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_150_175 = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_175_200 = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_200_300 = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_300_400 = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_400_500 = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_500_600 = CLNE_CLClimNE_log_data_proteases.copy()
CLClimNE_log_data_with_600 = CLNE_CLClimNE_log_data_proteases.copy()

# filter the data by the degradation rate:
CLClimNE_log_data_with_25_below = CLClimNE_log_data_with_25_below[CLClimNE_log_data_with_25_below['HL value'] <= 25]
CLClimNE_log_data_with_25_50 = CLClimNE_log_data_with_25_50[(CLClimNE_log_data_with_25_50['HL value'] > 25) & (CLClimNE_log_data_with_25_50['HL value'] <= 50)]
CLClimNE_log_data_with_50_75 = CLClimNE_log_data_with_50_75[(CLClimNE_log_data_with_50_75['HL value'] > 50) & (CLClimNE_log_data_with_50_75['HL value'] <= 75)]
CLClimNE_log_data_with_75_100 = CLClimNE_log_data_with_75_100[(CLClimNE_log_data_with_75_100['HL value'] > 75) & (CLClimNE_log_data_with_75_100['HL value'] <= 100)]
CLClimNE_log_data_with_100_125 = CLClimNE_log_data_with_100_125[(CLClimNE_log_data_with_100_125['HL value'] > 100) & (CLClimNE_log_data_with_100_125['HL value'] <= 125)]
CLClimNE_log_data_with_125_150 = CLClimNE_log_data_with_125_150[(CLClimNE_log_data_with_125_150['HL value'] > 125) & (CLClimNE_log_data_with_125_150['HL value'] <= 150)]
CLClimNE_log_data_with_150_175 = CLClimNE_log_data_with_150_175[(CLClimNE_log_data_with_150_175['HL value'] > 150) & (CLClimNE_log_data_with_150_175['HL value'] <= 175)]
CLClimNE_log_data_with_175_200 = CLClimNE_log_data_with_175_200[(CLClimNE_log_data_with_175_200['HL value'] > 175) & (CLClimNE_log_data_with_175_200['HL value'] <= 200)]
CLClimNE_log_data_with_200_300 = CLClimNE_log_data_with_200_300[(CLClimNE_log_data_with_200_300['HL value'] > 200) & (CLClimNE_log_data_with_200_300['HL value'] <= 300)]
CLClimNE_log_data_with_300_400 = CLClimNE_log_data_with_300_400[(CLClimNE_log_data_with_300_400['HL value'] > 300) & (CLClimNE_log_data_with_300_400['HL value'] <= 400)]
CLClimNE_log_data_with_400_500 = CLClimNE_log_data_with_400_500[(CLClimNE_log_data_with_400_500['HL value'] > 400) & (CLClimNE_log_data_with_400_500['HL value'] <= 500)]
CLClimNE_log_data_with_500_600 = CLClimNE_log_data_with_500_600[(CLClimNE_log_data_with_500_600['HL value'] > 500) & (CLClimNE_log_data_with_500_600['HL value'] <= 600)]
CLClimNE_log_data_with_600 = CLClimNE_log_data_with_600[CLClimNE_log_data_with_600['HL value'] > 600]



In [21]:
## Create scatter plot of CLNE counts to CLClim0NE counts, but color the protiens based on their protease assignment in the CLClim0NE model
# create the plot!
# TODO: do I need to normalize the data too ? and how do I get the plot to be an exact square? and the figure title centered?
def CLNE_CLClimNE_plot_by_half_life(fig, dataframe, CLNE_new_name, CLClimNE_new_name, protease=0):
    x = dataframe[CLNE_new_name]
    y = dataframe[CLClimNE_new_name]
    #hovertext = {dataframe["gene symbol"],dataframe["HL value"],dataframe["HL source"]}
    hovertext = dataframe.apply(lambda row: f"Common Name: {row['gene symbol']}<br>HL Value: {row['HL value']}<br>HL Source: {row['HL source']}<br>protease assignment: {row['protease assignment']}<br>CLNE AMC: {10**(row['Log10 CLNE_11192024 Average Monomer Counts'])}<br>CLClim3NE AMC: {10**(row['Log10 CLClim3NE1 Average Monomer Counts'])}", axis=1)

 
    # Add scatter trace
    if protease == 0: # no assignement
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"> 600 mins: ({np.shape(dataframe)[0]})", marker=dict(color='lightseagreen', size=3, opacity=.3)))

    elif protease == 1: # ClpP and Lon
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"500-600 mins: ({np.shape(dataframe)[0]})", marker=dict(color='cornflowerblue', size=5, opacity=.9)))
        
    elif protease == 2: # C L H
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"400-500 mins: ({np.shape(dataframe)[0]})", marker=dict(color='orange', size=4, opacity=.7)))
        
    elif protease == 4: # lon
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"300-400 mins: ({np.shape(dataframe)[0]})", marker=dict(color='yellowgreen', size=5, opacity=.9)))
        
    elif protease == 5: # clp
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"200-300 mins: ({np.shape(dataframe)[0]})", marker=dict(color='deeppink', size=4, opacity=.6)))
        
    elif protease == 6: # hslv
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"175-200 mins: ({np.shape(dataframe)[0]})", marker=dict(color='brown', size=5, opacity=.9)))
        
    elif protease == 7: # hslv
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"150-175 mins: ({np.shape(dataframe)[0]})", marker=dict(color='lightyellow', size=5, opacity=.9)))
        
    elif protease == 8: # hslv
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"125-150 mins: ({np.shape(dataframe)[0]})", marker=dict(color='yellow', size=5, opacity=.9)))
        
    elif protease == 9: # hslv
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"100-125 mins: ({np.shape(dataframe)[0]})", marker=dict(color='coral', size=5, opacity=.9)))
        
    elif protease == 10: # hslv
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"75-100 mins: ({np.shape(dataframe)[0]})", marker=dict(color='lawngreen', size=5, opacity=.9)))
        
    elif protease == 11: # hslv
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"50-75 mins: ({np.shape(dataframe)[0]})", marker=dict(color='orange', size=5, opacity=.9)))
        
    elif protease == 12: # hslv
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"25-50 mins: ({np.shape(dataframe)[0]})", marker=dict(color='darkorange', size=5, opacity=.9)))
        
    elif protease == 13: # hslv
        fig.add_trace(go.Scatter(x=x, y=y, hovertext=hovertext, mode='markers', name=f"< 25 mins: ({np.shape(dataframe)[0]})", marker=dict(color='red', size=5, opacity=.9)))

# now plot!

# Create figure
fig = go.Figure()

# Add scatter plots and trendlines for each dataset
CLNE_CLClimNE_plot_by_half_life(fig, CLClimNE_log_data_with_600,CLNE_new_name, CLClimNE_new_name, 0)
CLNE_CLClimNE_plot_by_half_life(fig, CLClimNE_log_data_with_500_600,CLNE_new_name, CLClimNE_new_name, 2)
CLNE_CLClimNE_plot_by_half_life(fig, CLClimNE_log_data_with_400_500,CLNE_new_name, CLClimNE_new_name, 1)

CLNE_CLClimNE_plot_by_half_life(fig, CLClimNE_log_data_with_300_400,CLNE_new_name, CLClimNE_new_name, 4)
CLNE_CLClimNE_plot_by_half_life(fig, CLClimNE_log_data_with_200_300,CLNE_new_name, CLClimNE_new_name, 5)
CLNE_CLClimNE_plot_by_half_life(fig, CLClimNE_log_data_with_175_200,CLNE_new_name, CLClimNE_new_name, 6)

CLNE_CLClimNE_plot_by_half_life(fig, CLClimNE_log_data_with_150_175,CLNE_new_name, CLClimNE_new_name, 7)
CLNE_CLClimNE_plot_by_half_life(fig, CLClimNE_log_data_with_125_150,CLNE_new_name, CLClimNE_new_name, 8)
CLNE_CLClimNE_plot_by_half_life(fig, CLClimNE_log_data_with_100_125,CLNE_new_name, CLClimNE_new_name, 9)
CLNE_CLClimNE_plot_by_half_life(fig, CLClimNE_log_data_with_75_100,CLNE_new_name, CLClimNE_new_name, 10)

CLNE_CLClimNE_plot_by_half_life(fig, CLClimNE_log_data_with_50_75,CLNE_new_name, CLClimNE_new_name, 11)
CLNE_CLClimNE_plot_by_half_life(fig, CLClimNE_log_data_with_25_50,CLNE_new_name, CLClimNE_new_name, 12)
CLNE_CLClimNE_plot_by_half_life(fig, CLClimNE_log_data_with_25_below,CLNE_new_name, CLClimNE_new_name, 13)

# add a y=x line
fig.add_trace(go.Scatter(x=[0, 6], y=[0, 6], mode="lines",
    line=go.scatter.Line(color="black", dash="dash"), opacity=0.2,
    name="y=x"));

# Update layout
fig.update_layout(
    title=f"Protein Counts from the New WCM plotted <br>against those from the Original WCM",
    xaxis_title=f"log10({CLNE_sequence} Protein Counts)",
    yaxis_title=f"log10({current_sequence} Protein Counts)",
    autosize=False,
    width=900,
    height=600, 
    showlegend=True,
    legend_title="Half Life Value Range")


# wont open in pycharm, so save as a html:
out_pth = "models/ecoli/analysis/local_notebooks/protease_substrate_relationships/protein_count_comparisons_outputs" 
out_pth = out_pth + "/" + current_sequence + "_vs_" + CLNE_sequence + "_protein_count_comparison_with_HLs_color_coded.html"
out_pth = os.path.expanduser(out_pth) 
output_dir = os.path.dirname(out_pth)
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it does not exist
fig.write_html(out_pth, auto_open=True) # displays the figure on a default web browser 



# next, do the same thing, but with the complexes highlighted as a symbol (like an x)